In [1]:
%pip install -q sqlite_vec
%pip install -q numpy

import sqlite3
import sqlite_vec
import hashlib
import json
from typing import List
import numpy as np

# Step 1 - Connection SQLite
conn = sqlite3.connect("demo_sqlitevec.db")
conn.enable_load_extension(True)    # Enable extension loading
sqlite_vec.load(conn)               # Load sqlite_vec extension
cur = conn.cursor()

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Create Table
cur.execute("""
CREATE TABLE IF NOT EXISTS docs (
    id INTEGER PRIMARY KEY,
    content TEXT NOT NULL
);
""")

# Create Vector Table
cur.execute("""
CREATE VIRTUAL TABLE IF NOT EXISTS vec_docs USING vec0(
    embedding FLOAT[8]  -- กำหนดว่าเวกเตอร์จะมี 8 มิติ
);
""")

In [3]:
def embed_text(text: str, dim: int = 8) -> List[float]:
    """
    function จำลองการสร้าง Embedding แบบง่ายๆ เพื่อการสาธิต
    ใช้ SHA256 ของข้อความเพื่อสร้างเวกเตอร์ที่ได้ผลลัพธ์เหมือนเดิมทุกครั้ง
    ของจริงต้องใช้โมเดล embedding ที่เหมาะสมกับงาน
    """
    h = hashlib.sha256(text.encode("utf-8")).digest()
    vec = []
    for i in range(dim):
        b1 = h[(i * 2) % len(h)]
        b2 = h[(i * 2 + 1) % len(h)]
        val = (b1 << 8) | b2
        f = (val / 65535.0) * 2.0 - 1.0  # ทำให้ค่าอยู่ในช่วง [-1, 1]
        vec.append(f)
    return vec


def decode_embedding(embedding):
    if isinstance(embedding, (bytes, bytearray)):
        try:
            vec = np.frombuffer(embedding, dtype=np.float32).tolist()
        except Exception:
            try:
                vec = json.loads(embedding.decode('utf-8'))
            except Exception:
                vec = repr(embedding)
    else:
        if isinstance(embedding, str):
            try:
                vec = json.loads(embedding)
            except Exception:
                vec = embedding
        else:
            vec = embedding
    return vec


In [4]:
# Insert docs only if table is empty (idempotent)
cur.execute("SELECT COUNT(*) FROM docs;")
docs = [
    (1, "The quick brown fox jumps over the lazy dog"),
    (2, "A fast auburn fox leaps above a sleepy canine"),
    (3, "An article about database systems and vector search"),
    (4, "Deep learning and embeddings for natural language processing"),
]
docs_count = cur.fetchone()[0]
if docs_count == 0:
    cur.executemany("INSERT INTO docs(id, content) VALUES (?, ?);", docs)

# Insert vector docs
cur.execute("SELECT COUNT(*) FROM vec_docs;")
vec_count = cur.fetchone()[0]
if vec_count == 0:
    rows = []
    for _id, text in docs:
        emb = embed_text(text, dim=8)
        rows.append((_id, json.dumps(emb)))

    cur.executemany("INSERT INTO vec_docs(rowid, embedding) VALUES (?, ?);", rows)
    conn.commit()

In [5]:
# Step 4 - Test query
query = "fox dog"
text_query = query
query_vec = embed_text(text_query, dim=8)
query_vec_json = json.dumps(query_vec)
res = cur.execute(
    """
    SELECT rowid, distance, embedding
    FROM vec_docs
    WHERE embedding MATCH ?
    ORDER BY distance
    LIMIT 2;
    """,
    (query_vec_json,)
).fetchall()

for rowid, distance, embedding in res:
    vec = decode_embedding(embedding)
    print(f"- rowid={rowid}  distance={float(distance):.12f} embedding={vec}")

- rowid=3  distance=1.915137887001 embedding=[-0.5786678791046143, -0.48274967074394226, -0.9851071834564209, 0.9522392749786377, 0.9597772359848022, 0.2738231420516968, 0.8145113587379456, -0.19572746753692627]
- rowid=1  distance=2.211798191071 embedding=[0.6848401427268982, 0.966430127620697, -0.9387502670288086, 0.004531929735094309, -0.17351034283638, 0.20888075232505798, 0.37526512145996094, -0.6382085680961609]
